![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/VOICE_OF_PATIENT.ipynb)

# **Voice of Patient-NER**

📌To run this yourself, you will need to upload your license keys to the notebook. Just Run The Cell Below in order to do that. Also You can open the file explorer on the left side of the screen and upload license_keys.json to the folder that opens. Otherwise, you can look at the example outputs at the bottom of the notebook.

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [ ]:
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql.types import StringType, IntegerType

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 4.4.4
Spark NLP_JSL Version : 4.4.4


## **`ner_vop`**

In [ ]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence_detector = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "en")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

clinical_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

ner_model = MedicalNerModel.pretrained("ner_vop", "en", "clinical/models")\
    .setInputCols(["sentence", "token","embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")

pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,
    tokenizer,
    clinical_embeddings,
    ner_model,
    ner_converter
    ])

sample_texts = ["Hi, I'm a 35-year-old male and have recently been diagnosed with diabetes. I've been feeling constant fatigue, excessive thirst, and frequent urination for the past few months. My doctor prescribed Metformin to manage my blood sugar levels but I've noticed some uncomfortable side effects like digestive issues and a metallic taste in my mouth. Will these effects subside over time?",
"Hello, I've been suffering from chronic back pain for the last 6 months. I visited a chiropractor who recommended physical therapy and exercises, but the pain persists. I've also been taking over-the-counter painkillers but they provide temporary relief at best. Can anyone suggest a more effective treatment?",
"I'm a 45-year-old woman dealing with insomnia for about a year now. It's affecting my daily life; I can't concentrate on work and it's causing me anxiety. I've tried melatonin and sleep hygiene practices but they haven't worked. My doctor prescribed Ambien, but I'm concerned about the potential for dependency. Can anyone share their experiences with this medication?",
"Hi everyone, I'm a 30-year-old female with a history of severe migraines. Recently, they've increased in frequency, and the pain is almost unbearable. My current medication isn't helping as much as it used to. Should I consider seeing a neurologist? Has anyone else experienced this?",
"I'm a 28-year-old male recently diagnosed with high cholesterol. I've been prescribed statins, but I've heard a lot about their side effects and I'm worried. I'm experiencing muscle aches and I'm not sure if it's due to the medication. Has anyone else experienced this? Any advice would be appreciated."]


data = spark.createDataFrame(sample_texts, StringType()).toDF("text")

result = pipeline.fit(data).transform(data)

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_vop download started this may take some time.
[OK!]


In [ ]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                     result.ner_chunk.begin,
                                     result.ner_chunk.end,
                                     result.ner_chunk.metadata)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("ner_label"),
              F.expr("cols['3']['confidence']").alias("confidence")).show(30, truncate=False)

+------------------+-----+---+----------+----------+
|chunk             |begin|end|ner_label |confidence|
+------------------+-----+---+----------+----------+
|35-year-old       |10   |20 |Age       |0.9819    |
|male              |22   |25 |Gender    |0.9993    |
|recently          |36   |43 |DateTime  |0.9781    |
|diabetes          |65   |72 |Disease   |0.9691    |
|constant          |93   |100|Modifier  |0.985     |
|fatigue           |102  |108|Symptom   |0.9906    |
|excessive         |111  |119|Modifier  |0.8801    |
|thirst            |121  |126|Symptom   |0.5784    |
|frequent urination|133  |150|Symptom   |0.6457    |
|few months        |165  |174|Duration  |0.60105   |
|doctor            |180  |185|Employment|0.9914    |
|Metformin         |198  |206|Drug      |0.9943    |
|blood sugar levels|221  |238|Test      |0.57329994|
|digestive         |294  |302|BodyPart  |0.9       |
|metallic taste    |317  |330|Symptom   |0.6867    |
|mouth             |338  |342|BodyPart  |0.876

In [ ]:
from sparknlp_display import NerVisualizer

for i in range(len(sample_texts)):
    NerVisualizer().display(
        result = result.collect()[i],
        label_col = 'ner_chunk',
        document_col = 'document')
    print("\n"*2)


## **`ner_vop_anatomy`**

In [ ]:
ner_model = MedicalNerModel.pretrained("ner_vop_anatomy", "en", "clinical/models")\
    .setInputCols(["sentence", "token","embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")


pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,
    tokenizer,
    clinical_embeddings,
    ner_model,
    ner_converter
    ])

sample_texts = ["Greetings, I'm a 28-year-old woman who's been experiencing severe abdominal pain and bloating. I was diagnosed with endometriosis, and my doctor suggested laparoscopic surgery. Anyone who has gone through this, could you share your recovery experience?",
"Hello everyone, I'm a 40-year-old man who has been suffering from severe shoulder pain. I was diagnosed with a rotator cuff tear, and my doctor recommended physiotherapy as a first approach. Anyone out there who has had a similar experience? Any suggestions on effective exercises?",
"Hey guys, I'm a 55-year-old woman who has been having trouble with my vision. My ophthalmologist diagnosed me with early-stage glaucoma, and prescribed eye drops for the pressure. Anyone else dealing with this? How do you cope with the diagnosis?"]

data = spark.createDataFrame(sample_texts, StringType()).toDF("text")

result = pipeline.fit(data).transform(data)

ner_vop_anatomy download started this may take some time.
[OK!]


In [ ]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                     result.ner_chunk.begin,
                                     result.ner_chunk.end,
                                     result.ner_chunk.metadata)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("ner_label"),
              F.expr("cols['3']['confidence']").alias("confidence")).show(30, truncate=False)

+---------+-----+---+---------+----------+
|chunk    |begin|end|ner_label|confidence|
+---------+-----+---+---------+----------+
|abdominal|66   |74 |BodyPart |0.9996    |
|shoulder |73   |80 |BodyPart |0.9996    |
|eye      |152  |154|BodyPart |0.9951    |
+---------+-----+---+---------+----------+



In [ ]:
from sparknlp_display import NerVisualizer

for i in range(len(sample_texts)):
    NerVisualizer().display(
        result = result.collect()[i],
        label_col = 'ner_chunk',
        document_col = 'document')
    print("\n"*2)


## **`ner_vop_demographic`**

In [ ]:
ner_model = MedicalNerModel.pretrained("ner_vop_demographic", "en", "clinical/models")\
    .setInputCols(["sentence", "token","embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")

pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,
    tokenizer,
    clinical_embeddings,
    ner_model,
    ner_converter
    ])

sample_texts = ["Hello folks, I'm a 30-year-old woman battling chronic insomnia. I used to consume caffeine daily, but I've stopped since my condition worsened. The medication my doctor prescribed doesn't seem to help, and the side effects are too much. Has anyone faced similar issues? What alternatives could I explore for better sleep? Any suggestions for handling insomnia without depending on medication?",
"Hi everyone, I'm a 26-year-old man grappling with persistent knee pain. I used to be a long-distance runner, but I've had to cut back significantly due to the pain. I've been trying various meds, but none have proven effective so far. Has anyone else encountered this? What other methods could I use for pain management? Do you have any non-medication tips for dealing with such issues?",
"Hey there, I'm a 34-year-old woman dealing with severe migraines. I don't smoke or drink. I used to work night shifts, but I quit a year ago to improve my health. I've tried several medications, but the side effects are pretty harsh. Has anyone else experienced this? What else could I try for migraine relief? Any advice on managing migraines without medication would be greatly appreciated.",
"Greetings everyone, I'm a 37-year-old man who's struggling with terrible acid reflux. I used to consume fast food regularly, but I've drastically changed my diet since this problem started. However, the medication I'm taking isn't helping much. Has anyone faced similar issues? What else can I do to manage acid reflux? I would appreciate any tips for dealing with this without relying on medication.",
"Hi folks, I'm a 29-year-old woman dealing with serious asthma issues. I used to live in a polluted city, but I moved to a cleaner environment a few years ago. I'm on medication, but it doesn't seem to alleviate my condition effectively. Has anyone else had this experience? What else could I try for asthma relief? Any non-medication tips for managing asthma would be really helpful."]

data = spark.createDataFrame(sample_texts, StringType()).toDF("text")

result = pipeline.fit(data).transform(data)

ner_vop_demographic download started this may take some time.
[OK!]


In [ ]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                     result.ner_chunk.begin,
                                     result.ner_chunk.end,
                                     result.ner_chunk.metadata)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("ner_label"),
              F.expr("cols['3']['confidence']").alias("confidence")).show(30, truncate=False)

+-----------+-----+---+----------+----------+
|chunk      |begin|end|ner_label |confidence|
+-----------+-----+---+----------+----------+
|30-year-old|19   |29 |Age       |0.9918    |
|woman      |31   |35 |Gender    |0.9999    |
|caffeine   |82   |89 |Substance |0.9782    |
|doctor     |162  |167|Employment|0.9918    |
|26-year-old|19   |29 |Age       |0.9891    |
|man        |31   |33 |Gender    |0.9986    |
|34-year-old|17   |27 |Age       |0.9954    |
|woman      |29   |33 |Gender    |0.9999    |
|smoke      |74   |78 |Substance |0.9926    |
|drink      |83   |87 |Substance |0.9894    |
|37-year-old|26   |36 |Age       |0.9905    |
|man        |38   |40 |Gender    |0.9985    |
|29-year-old|16   |26 |Age       |0.9928    |
|woman      |28   |32 |Gender    |0.9999    |
+-----------+-----+---+----------+----------+



In [ ]:
from sparknlp_display import NerVisualizer

for i in range(len(sample_texts)):
    NerVisualizer().display(
        result = result.collect()[i],
        label_col = 'ner_chunk',
        document_col = 'document')
    print("\n"*2)


## **`ner_vop_clinical_dept`**

In [ ]:
ner_model = MedicalNerModel.pretrained("ner_vop_clinical_dept", "en", "clinical/models")\
    .setInputCols(["sentence", "token","embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")

pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,
    tokenizer,
    clinical_embeddings,
    ner_model,
    ner_converter
    ])

sample_texts = ["Hello all! I'm a 38-year-old man and I've just been discharged from the gastroenterology department at the hospital. I was hospitalized due to a severe bout of ulcerative colitis and had to undergo a procedure to remove part of my colon. I'm feeling quite anxious about my recovery process. Does anyone have any advice on diet changes or general tips for dealing with colitis at home? Also, should I be worried about future flare-ups?",
"Hey guys! I'm a 50-year-old woman and I'm in the neurology department receiving treatment for multiple sclerosis. I've just started my course of disease-modifying therapy and it's quite tough. I'm feeling fatigued and dealing with some muscle weakness, but the doctors say it's normal. Can anyone offer tips on managing these side effects?",
"Greetings everyone! I'm a 24-year-old nonbinary individual who was recently diagnosed with rheumatoid arthritis. I'm still at the hospital learning to manage the pain and adapt to my new medication regimen. I'm feeling a bit overwhelmed and could use some advice. Does anyone have tips for dealing with flare-ups and keeping track of all the medications?",
"Hi folks! I'm a 60-year-old man and I just got discharged from the ophthalmology department after having cataract surgery. The recovery process is somewhat challenging and my vision is still quite blurry. I've been given some protective eyewear, but it's quite uncomfortable. Any suggestions for making the eyewear more comfortable or tips on adjusting to life post-surgery?",
"Hello everyone! I'm a 35-year-old woman and I've just been diagnosed with asthma. I'm in the pulmonary department at the hospital, and they're showing me how to use an inhaler and manage my symptoms. The thought of having an asthma attack is pretty nerve-wracking. Does anyone else have experience managing asthma? Any advice for adjusting to life with a chronic respiratory condition?"]

data = spark.createDataFrame(sample_texts, StringType()).toDF("text")

result = pipeline.fit(data).transform(data)

ner_vop_clinical_dept download started this may take some time.
[OK!]


In [ ]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                     result.ner_chunk.begin,
                                     result.ner_chunk.end,
                                     result.ner_chunk.metadata)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("ner_label"),
              F.expr("cols['3']['confidence']").alias("confidence")).show(30, truncate=False)

+---------------------------+-----+---+------------------+----------+
|chunk                      |begin|end|ner_label         |confidence|
+---------------------------+-----+---+------------------+----------+
|discharged                 |52   |61 |AdmissionDischarge|0.9989    |
|gastroenterology department|72   |98 |ClinicalDept      |0.845     |
|hospital                   |107  |114|ClinicalDept      |0.9909    |
|hospitalized               |123  |134|AdmissionDischarge|0.9984    |
|neurology department       |49   |68 |ClinicalDept      |0.8623    |
|hospital                   |130  |137|ClinicalDept      |0.9897    |
|discharged                 |47   |56 |AdmissionDischarge|0.9957    |
|ophthalmology department   |67   |90 |ClinicalDept      |0.88530004|
|pulmonary department       |93   |112|ClinicalDept      |0.8076    |
|hospital                   |121  |128|ClinicalDept      |0.9907    |
+---------------------------+-----+---+------------------+----------+



In [ ]:
from sparknlp_display import NerVisualizer

for i in range(len(sample_texts)):
    NerVisualizer().display(
        result = result.collect()[i],
        label_col = 'ner_chunk',
        document_col = 'document')
    print("\n"*2)


## **`ner_vop_problem_reduced`**

In [ ]:
ner_model = MedicalNerModel.pretrained("ner_vop_problem_reduced", "en", "clinical/models")\
    .setInputCols(["sentence", "token","embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")

pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,
    tokenizer,
    clinical_embeddings,
    ner_model,
    ner_converter
    ])

sample_texts = ["Hello all, I'm a 45-year-old man who has been experiencing headaches and blurring of vision lately. My doctor ordered some tests, and it appears I have glaucoma. He recommended that I start using prescribed eye drops and make some lifestyle changes to prevent further vision loss. Has anyone else been diagnosed with glaucoma? Any advice on managing the condition?",
"Hey guys, I'm a 55-year-old woman who has been feeling fatigued and experiencing unintentional weight loss. My doctor ordered some tests, and it turns out that I have type 2 diabetes. She suggested that I start taking medication, adjust my diet and include regular exercise to manage my blood sugar levels. Anyone else dealing with diabetes? Any tips for managing the symptoms?",
"Hi everyone, I'm a 42-year-old man who has been experiencing discomfort and tightness in my lower back. After a series of tests, my doctor diagnosed me with chronic lower back pain and recommended that I start physiotherapy and take prescribed medication for pain relief. Has anyone else dealt with chronic lower back pain? Any advice on easing the discomfort?",
"Hello, I'm a 38-year-old woman who has been having difficulty sleeping and persistent worrying. My doctor diagnosed me with generalized anxiety disorder and recommended that I start taking medication, practicing mindfulness, and consider talking to a counselor. Has anyone else experienced this? Any tips for managing anxiety symptoms?",
"Hi there, I'm a 50-year-old man who has been feeling extremely tired and has developed itchy skin recently. My doctor ran some tests and diagnosed me with chronic kidney disease. He recommended that I start on medication, monitor my diet, and lead a healthier lifestyle to manage the condition. Any fellow chronic kidney disease patients here? Any advice for managing the symptoms and keeping up with the lifestyle changes?"]

data = spark.createDataFrame(sample_texts, StringType()).toDF("text")

result = pipeline.fit(data).transform(data)

ner_vop_problem_reduced download started this may take some time.
[OK!]


In [ ]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                     result.ner_chunk.begin,
                                     result.ner_chunk.end,
                                     result.ner_chunk.metadata)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("ner_label"),
              F.expr("cols['3']['confidence']").alias("confidence")).show(30, truncate=False)

+----------------------------+-----+---+---------+----------+
|chunk                       |begin|end|ner_label|confidence|
+----------------------------+-----+---+---------+----------+
|headaches                   |59   |67 |Problem  |0.9959    |
|blurring of vision          |73   |90 |Problem  |0.90063334|
|glaucoma                    |152  |159|Problem  |0.9802    |
|vision loss                 |268  |278|Problem  |0.76755   |
|glaucoma                    |317  |324|Problem  |0.9807    |
|fatigued                    |55   |62 |Problem  |0.9862    |
|unintentional weight loss   |81   |105|Problem  |0.7252    |
|type 2 diabetes             |167  |181|Problem  |0.5654    |
|diabetes                    |332  |339|Problem  |0.9787    |
|discomfort                  |61   |70 |Problem  |0.9944    |
|tightness                   |76   |84 |Problem  |0.9932    |
|chronic                     |157  |163|Modifier |0.9966    |
|pain                        |176  |179|Problem  |0.9856    |
|pain   

In [ ]:
from sparknlp_display import NerVisualizer

for i in range(len(sample_texts)):
    NerVisualizer().display(
        result = result.collect()[i],
        label_col = 'ner_chunk',
        document_col = 'document')
    print("\n"*2)


## **`ner_vop_problem`**

In [ ]:
ner_model = MedicalNerModel.pretrained("ner_vop_problem", "en", "clinical/models")\
    .setInputCols(["sentence", "token","embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")

pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,
    tokenizer,
    clinical_embeddings,
    ner_model,
    ner_converter
    ])

sample_texts = ["Hello everyone, I'm a 45-year-old man who has been experiencing severe headaches and sensitivity to light. My doctor ran some tests and it turns out I have migraines. She recommended I take certain medications and alter my diet to help prevent them. Does anyone else suffer from migraines? Any advice for preventing them?",
"Hey all, I'm a 55-year-old woman who has been dealing with excessive thirst and frequent urination. My doctor ordered some tests and concluded that I have type 2 diabetes. She advised that I should start taking medication and commit to a regular exercise routine. Has anyone else been diagnosed with type 2 diabetes? Any tips on managing blood sugar levels?",
"Greetings, I'm a 50-year-old man who has been experiencing some rapid weight gain and fatigue. My doctor diagnosed me with hypothyroidism and recommended I start taking thyroid medication and focus on a balanced diet. Has anyone else been diagnosed with hypothyroidism? Any tips for managing the symptoms?",
"Hello there, I'm a 40-year-old woman who has been feeling consistently tired and has pale skin. My doctor ran some tests, and I've been diagnosed with iron-deficiency anemia. She recommended that I start iron supplements and eat iron-rich foods. Anyone else dealing with anemia? Any tips to increase iron levels in the body?",
"Hi everyone, I'm a 60-year-old man who has been experiencing tremors and stiffness in my movements. My doctor diagnosed me with Parkinson's disease and recommended I start medication and physiotherapy to manage my symptoms. Has anyone else been diagnosed with Parkinson's? Any tips for handling the symptoms?"]

data = spark.createDataFrame(sample_texts, StringType()).toDF("text")

result = pipeline.fit(data).transform(data)

ner_vop_problem download started this may take some time.
[OK!]


In [ ]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                     result.ner_chunk.begin,
                                     result.ner_chunk.end,
                                     result.ner_chunk.metadata)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("ner_label"),
              F.expr("cols['3']['confidence']").alias("confidence")).show(30, truncate=False)

+----------------------+-----+---+---------+----------+
|chunk                 |begin|end|ner_label|confidence|
+----------------------+-----+---+---------+----------+
|severe                |64   |69 |Modifier |0.9992    |
|headaches             |71   |79 |Symptom  |0.9838    |
|sensitivity to light  |85   |104|Symptom  |0.8663333 |
|migraines             |156  |164|Disease  |0.995     |
|migraines             |279  |287|Disease  |0.9915    |
|excessive             |59   |67 |Modifier |0.9699    |
|thirst                |69   |74 |Symptom  |0.4964    |
|frequent urination    |80   |97 |Symptom  |0.7424    |
|type 2 diabetes       |155  |169|Disease  |0.59613335|
|type 2 diabetes       |300  |314|Disease  |0.5729333 |
|rapid                 |64   |68 |Modifier |0.6887    |
|weight gain           |70   |80 |Symptom  |0.6422    |
|fatigue               |86   |92 |Symptom  |0.9864    |
|hypothyroidism        |123  |136|Disease  |0.9037    |
|hypothyroidism        |254  |267|Disease  |0.96

In [ ]:
from sparknlp_display import NerVisualizer

for i in range(len(sample_texts)):
    NerVisualizer().display(
        result = result.collect()[i],
        label_col = 'ner_chunk',
        document_col = 'document')
    print("\n"*2)


## **`ner_vop_temporal`**

In [ ]:
ner_model = MedicalNerModel.pretrained("ner_vop_temporal", "en", "clinical/models")\
    .setInputCols(["sentence", "token","embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")

pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,
    tokenizer,
    clinical_embeddings,
    ner_model,
    ner_converter
    ])

sample_texts = ["Hello everyone, I'm a 40-year-old man who was recently diagnosed with anxiety. I've been seeing a mental health professional and taking prescribed medication for the past four months, and I'm starting to regain my inner peace. I attend counseling sessions twice a month and follow my medication schedule rigorously. I've noticed I feel calmer in the afternoons compared to the evenings. Has anyone else experienced the same? Any suggestions for managing anxiety in the long run?",
"Greetings all, I'm a 33-year-old woman who has embarked on a new health journey. I've committed to yoga and meditation three times a week for the past three months, and I'm witnessing positive transformations. I spend an hour each session focusing on my breath and postures. Besides this, I've started following a balanced diet and reduced my caffeine intake. Have any of you made similar lifestyle changes? How long did it take for you to see significant improvements? Any advice for maintaining the momentum?",
"Hello folks, I'm a 50-year-old man who recently battled a severe flu. I was ill for around ten days, and it was incredibly tough. Despite persistent headaches and weakness, I ensured ample rest and hydration, but it was quite an ordeal. Has anyone else experienced a challenging flu recently? Do you have any tips for managing the symptoms and hastening the recovery?",
"Hi everyone, I'm a 26-year-old woman who's been struggling with chronic migraines since childhood. They seem to get worse during changes in weather, and can last for several days. I've tried various medications and natural remedies, but finding a perfect solution is still a challenge. I try to avoid triggers such as bright lights and loud noises, but it's not always feasible. Has anyone else been battling migraines for a long time? Any advice for managing the symptoms or preventing the attacks?",
"Hey all, I'm a 29-year-old man grappling with restlessness at night for the past six months. Falling asleep is a struggle, and maintaining a sound sleep seems next to impossible. Despite trying several sleep techniques and maintaining a regular sleep schedule, I've had little success. I've also attempted to stay away from screens before bedtime, but it's easier said than done. Has anyone else been facing similar sleep disturbances? Any suggestions for long-term management of such sleep disorders?"]

data = spark.createDataFrame(sample_texts, StringType()).toDF("text")

result = pipeline.fit(data).transform(data)

ner_vop_temporal download started this may take some time.
[OK!]


In [ ]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                     result.ner_chunk.begin,
                                     result.ner_chunk.end,
                                     result.ner_chunk.metadata)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("ner_label"),
              F.expr("cols['3']['confidence']").alias("confidence")).show(30, truncate=False)

+------------------+-----+---+---------+----------+
|chunk             |begin|end|ner_label|confidence|
+------------------+-----+---+---------+----------+
|recently          |46   |53 |DateTime |0.9763    |
|four months       |171  |181|Duration |0.72395   |
|twice a month     |256  |268|Frequency|0.8692    |
|in the afternoons |343  |359|DateTime |0.5302    |
|three times a week|119  |136|Frequency|0.830925  |
|three months      |151  |162|Duration |0.7215    |
|an hour           |218  |224|Duration |0.48605   |
|recently          |39   |46 |DateTime |0.9757    |
|ten days          |91   |98 |Duration |0.77295   |
|recently          |283  |290|DateTime |0.978     |
|several days      |166  |177|Duration |0.69624996|
|always            |362  |367|Frequency|0.9751    |
|for a long time   |419  |433|Duration |0.622875  |
|at night          |59   |66 |DateTime |0.46045   |
|six months        |81   |90 |Duration |0.7422    |
|before bedtime    |332  |345|DateTime |0.65015   |
+-----------

In [ ]:
from sparknlp_display import NerVisualizer

for i in range(len(sample_texts)):
    NerVisualizer().display(
        result = result.collect()[i],
        label_col = 'ner_chunk',
        document_col = 'document')
    print("\n"*2)


## **`ner_vop_test`**

In [ ]:
ner_model = MedicalNerModel.pretrained("ner_vop_test", "en", "clinical/models")\
    .setInputCols(["sentence", "token","embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")

pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,
    tokenizer,
    clinical_embeddings,
    ner_model,
    ner_converter
    ])

sample_texts = ["Hi there, I'm a 45-year-old man who has been noticing frequent bouts of dizziness recently. My doctor ordered some tests and my blood pressure turned out to be high. He suggested that I start taking medication and make some dietary changes to manage my blood pressure. Has anyone else experienced high blood pressure? Any advice on coping with it?",
"Hello folks, I'm a 38-year-old woman who has been having irregular periods and sudden weight gain. My doctor conducted a few tests, and my hormone levels were found to be imbalanced. She suggested that I should start hormonal therapy. Does anyone here have experience with hormonal imbalances? How effective was the treatment for you?",
"Hey there, I'm a 48-year-old man who's been dealing with severe fatigue and muscle weakness. My doctor ordered some tests, and it turns out I have low testosterone levels. He suggested I start testosterone replacement therapy. Has anyone else faced low testosterone issues? How long did it take you to feel a difference after starting therapy?"]

data = spark.createDataFrame(sample_texts, StringType()).toDF("text")

result = pipeline.fit(data).transform(data)

ner_vop_test download started this may take some time.
[OK!]


In [ ]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                     result.ner_chunk.begin,
                                     result.ner_chunk.end,
                                     result.ner_chunk.metadata)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("ner_label"),
              F.expr("cols['3']['confidence']").alias("confidence")).show(30, truncate=False)

+--------------+-----+---+----------+----------+
|chunk         |begin|end|ner_label |confidence|
+--------------+-----+---+----------+----------+
|blood pressure|128  |141|VitalTest |0.78945   |
|high          |160  |163|TestResult|0.8991    |
|blood pressure|253  |266|VitalTest |0.81445   |
|high          |297  |300|TestResult|0.9356    |
|blood pressure|302  |315|VitalTest |0.7602    |
|hormone levels|139  |152|Test      |0.7985    |
|low           |147  |149|TestResult|0.8636    |
|testosterone  |151  |162|Test      |0.9883    |
|low           |249  |251|TestResult|0.832     |
|testosterone  |253  |264|Test      |0.9131    |
+--------------+-----+---+----------+----------+



In [ ]:
from sparknlp_display import NerVisualizer

for i in range(len(sample_texts)):
    NerVisualizer().display(
        result = result.collect()[i],
        label_col = 'ner_chunk',
        document_col = 'document')
    print("\n"*2)


## **`ner_vop_treatment`**

In [ ]:
ner_model = MedicalNerModel.pretrained("ner_vop_treatment", "en", "clinical/models")\
    .setInputCols(["sentence", "token","embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")

pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,
    tokenizer,
    clinical_embeddings,
    ner_model,
    ner_converter
    ])

sample_texts = ["Hello all, I'm a 62-year-old man who was diagnosed with type 2 diabetes last year. My doctor prescribed me 500mg of Metformin, which I take twice a day. I've seen my blood sugar levels stabilize, but I've also experienced some stomach upset. Anyone else on Metformin? How long did it take for you to see consistent results? Any tips for dealing with the side effects?",
"Hi everyone, I'm a 48-year-old woman who has been dealing with chronic arthritis pain. My doctor recently started me on 200mg of Celebrex, which I take once a day. It helps with the inflammation, but leaves me with some heartburn and indigestion. Has anyone else taken Celebrex for arthritis? Any advice for managing the side effects or additional pain management suggestions?",
"Hey, I'm a 37-year-old man who was recently diagnosed with ADHD. My doctor prescribed me 30mg of Adderall, which I take daily in the morning. While it's helped me to focus and be more productive, I've been having trouble sleeping and a decreased appetite. Anyone else on Adderall? How long did it take for you to adjust? Any tips for managing the side effects?",
"Greetings, I'm a 51-year-old woman who has been struggling with depression. I was prescribed 20mg of Prozac daily by my doctor. My mood has improved, but I've been experiencing a bit of nausea and dry mouth. Has anyone else taken Prozac for depression? How long until you saw a significant improvement? Any tips for managing the side effects?",
"Hello there, I'm a 44-year-old guy who's been dealing with high cholesterol levels. I've been taking 20mg of Lipitor each night as per my doctor's prescription. I'm seeing a drop in my cholesterol, but I'm also feeling a bit of muscle pain. Has anyone else taken Lipitor? How long did it take for your levels to normalize? Any advice on dealing with side effects?"]

data = spark.createDataFrame(sample_texts, StringType()).toDF("text")

result = pipeline.fit(data).transform(data)

ner_vop_treatment download started this may take some time.
[OK!]


In [ ]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                     result.ner_chunk.begin,
                                     result.ner_chunk.end,
                                     result.ner_chunk.metadata)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("ner_label"),
              F.expr("cols['3']['confidence']").alias("confidence")).show(30, truncate=False)

+-----------+-----+---+---------+----------+
|chunk      |begin|end|ner_label|confidence|
+-----------+-----+---+---------+----------+
|500mg      |107  |111|Dosage   |0.9958    |
|Metformin  |116  |124|Drug     |0.9832    |
|twice a day|140  |150|Frequency|0.8087    |
|Metformin  |257  |265|Drug     |0.9802    |
|200mg      |120  |124|Dosage   |0.9913    |
|Celebrex   |129  |136|Drug     |0.9974    |
|once a day |152  |161|Frequency|0.80850005|
|Celebrex   |269  |276|Drug     |0.9991    |
|30mg       |89   |92 |Dosage   |0.989     |
|Adderall   |97   |104|Drug     |0.9996    |
|daily      |120  |124|Frequency|0.9499    |
|Adderall   |271  |278|Drug     |0.9996    |
|20mg       |93   |96 |Dosage   |0.9984    |
|Prozac     |101  |106|Drug     |0.9975    |
|daily      |108  |112|Frequency|0.9861    |
|Prozac     |230  |235|Drug     |0.9992    |
|20mg       |101  |104|Dosage   |0.9987    |
|Lipitor    |109  |115|Drug     |0.9989    |
|each night |117  |126|Frequency|0.89400005|
|Lipitor  

In [ ]:
from sparknlp_display import NerVisualizer

for i in range(len(sample_texts)):
    NerVisualizer().display(
        result = result.collect()[i],
        label_col = 'ner_chunk',
        document_col = 'document')
    print("\n"*2)

# **Voice of Patient-Classification**

## **`bert_sequence_classifier_vop_side_effect`**

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

sequenceClassifier = MedicalBertForSequenceClassification.pretrained("bert_sequence_classifier_vop_side_effect", "en", "clinical/models")\
    .setInputCols(["document",'token'])\
    .setOutputCol("prediction")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    sequenceClassifier
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

bert_sequence_classifier_vop_side_effect download started this may take some time.
[OK!]


In [ ]:
sample_texts = ["""Hello everyone, I started using this drug called "NerveCalm." It was supposed to help me with my nerves, but it made me so sleepy I couldn't function. I was dozing off at work, and even my morning coffee couldn't shake off the drowsiness. This wasn't the calmness I was aiming for, you know?""",
"""Hello all, my doctor prescribed me a drug called "Zonex" to help manage my insomnia. But instead of a peaceful sleep, it gave me weird, vivid dreams all night. I woke up feeling as if I hadn't slept at all. Not the restful night's sleep I had in mind, you understand?""",
"""Hey everyone, I've been taking a medication called "LiverLight" to manage my liver function, and I'm delighted to share that there are no side effects. It's been doing a great job at detoxifying my body without causing any disturbances. I'm truly grateful to have discovered a medication that works so well for me.""",
"""Greetings everyone, I've recently started taking a pill called "BrainBooster" to help with my memory and cognition. Guess what? Zero side effects for me! It has been quite helpful in improving my memory. I feel like I'm acing my tasks and being productive without any setbacks. It's comforting to finally find a drug that delivers without any complications."""]

In [ ]:
light_model = LightPipeline(model)
result = light_model.fullAnnotate(sample_texts)

In [ ]:
for i in range(len(sample_texts)):
    r = result[i]['prediction'][0]
    print(pd.DataFrame({'prediction': [[r.begin, r.end, r.result, r.metadata]]}))


                                            prediction
0  [0, 290, True, [sentence, Some(False), Some(True)]]
                                            prediction
0  [0, 266, True, [sentence, Some(False), Some(True)]]
                                             prediction
0  [0, 313, False, [sentence, Some(False), Some(True)]]
                                             prediction
0  [0, 356, False, [sentence, Some(False), Some(True)]]
